In [25]:
!pip install pyspark
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
import pandas as pd
from nltk.corpus import stopwords
import re

# load data
#df = pd.read_csv('./jobs_small.csv', encoding="latin-1")
df = pd.read_csv('./jobs.csv', encoding="utf-8")
#print(df.head())

# text preprocessing
REPLACE_BY_SPACE_RE = re.compile('[#+_/(){}!^?<>"''*\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
match_regex = re.compile('\d+')
STOPWORDS = set(stopwords.words('english'))

# data cleaning
def clean_text(text):
    # change to lower-csae
    text = str(text).lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    # remove BAD_SYMBOLS_RE
    text = BAD_SYMBOLS_RE.sub('', text)
    text = match_regex.sub('', text)
    # drop the stopwords
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text

# read and clean the resume file
f = open('./CE.txt', 'r')          ############# change resume here ####################
text = f.read()
text = clean_text(text)

# clean the desc field
df['desc_clean'] = df['description'].apply(clean_text)
df.drop(columns=['description', 'id'], inplace=True)
df.loc[0] = ['resume', 0, 0, 0, text]

for i in range(len(df)):
  try:
    if df['desc_clean'][i]=='nan' or df['desc_clean'][i]=='' or len(df['desc_clean'][i]) < 100:
      df.drop(labels=i, inplace=True)
  except:
    continue

df.dropna(axis=0, inplace=True)
df['id'] = [i for i in range(len(df))]
#print(df['desc_clean'])
df.to_csv('./jobs_clean.csv')

In [27]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
#from pyspark.ml.feature import NGram

spark=SparkSession \
        .builder \
        .appName('tfidf_app') \
        .getOrCreate()

# load data
df0 = spark.read.csv("./jobs_clean.csv", header=True, multiLine=True, inferSchema=True)
df1 = pd.read_csv('./jobs_clean.csv')
#df0.show()
print('The number of jobs：',df0.count())
print('\nthe distinct jobs name: ', df1.job.unique())
print('\nThere are', len(df1.job.unique())-1, 'different kinds of jobs in the table.')

# split the desc field
tokenizer = Tokenizer(inputCol='desc_clean', outputCol='desc_words')
df = tokenizer.transform(df0)
#df.show()
#df.select('desc_words').show(10)

# compute TF-IDF
hashingTF = HashingTF(inputCol='desc_words', outputCol='desc_words_tf')
tf = hashingTF.transform(df).cache()
idf = IDF(inputCol='desc_words_tf', outputCol='desc_words_tfidf').fit(tf)
tfidf = idf.transform(tf).cache()
#print('tfidf for each job:', tfidf.select('desc_words_tfidf').show(10,truncate=False))

# data normalization
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="desc_words_tfidf", outputCol="norm")
tfidf = normalizer.transform(tfidf)
#tfidf.select("id", "norm").show(6)

# compute similarity between jobs and resume
import pyspark.sql.functions as psf 
from pyspark.sql.types import DoubleType
print('\nCompute the similarity between jobs and resume...')
dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType()) # define dot-product function
tfidf = tfidf.alias("a1").join(tfidf.alias("a2"), psf.col("a1.id") == 0)\
        .select(
            psf.col("a1.job"),
            psf.col("a1.id").alias("id1"), 
            psf.col("a2.id").alias("id2"), 
            dot_udf("a1.norm", "a2.norm").alias("similarity"))
#tfidf.show(10)
print('Done!')

The number of jobs： 14791

the distinct jobs name:  ['resume' 'data-scientist' 'software-developer' 'statistician'
 'IT manager' 'web developer' 'database administrator'
 'information security analyst' 'computer systems analyst'
 'computer network architect' 'computer support specialist'
 'Circuit Design Engineer' 'FPGA Engineer' 'Embedded Systems Engineer'
 'Electrical Design Engineer' 'Telecommunications Engineer'
 'Machine Learning Engineer' 'python' 'software engineer'
 'Java Software Engineer' 'Python Software Engineer'
 'Golang Software Engineer' 'PHP developer' 'Node js developer'
 'Ruby developer' 'Front End Web Developer' 'SQL Developer'
 'flash Developer' 'JavaScript Developer' 'Android Developer'
 'IOS Developer' 'Database Engineer' 'Spark Engineer' 'Data Analyst'
 'electrical test' 'Performance Test Engineer' 'Test Automation Engineer'
 'PLC Technician' 'DSP engineer' 'ARM engineer' 'NLP engineer'
 'computer vision engineer' 'Operations Technician'
 'Software Product Manage

In [28]:
# show Top-20 matched jobs
match = tfidf.where('id1 = 0').sort('similarity', ascending=False).where('id2 > 0')
top_match = match.limit(20)
print('Top 20 matched jobs:')
df0.alias("a1").join(top_match.alias("a2"), psf.col("a1.id") == psf.col("a2.id2"))\
    .select(psf.col("a1.job"), "a1.company", "a1.location", "a2.similarity")\
    .sort('similarity', ascending=False).show()

Top 20 matched jobs:
+--------------------+--------------------+--------------------+-------------------+
|                 job|             company|            location|         similarity|
+--------------------+--------------------+--------------------+-------------------+
|        NLP engineer|         Axiom Group|       Austin, Texas| 0.1887218973121401|
|      data-scientist|      KR Elixir, Inc|San Francisco, Ca...| 0.1818072658180541|
|        NLP engineer| Tailored Management|South San Francis...|0.18022456541360377|
|Machine Learning ...|         CyberCoders|San Francisco, Ca...|0.16317681882956594|
|        NLP engineer|Whitney, Bradley ...|Crystal City, Vir...|0.15284096175042325|
|        NLP engineer|     PayPal Holdings|San Jose, Califor...|0.15275969005591425|
|      Spark Engineer|     PayPal Holdings|San Jose, Califor...|0.15275969005591425|
|        NLP engineer|              Leidos|Washington, Distr...| 0.1505508492704159|
|Python Software E...|IVYSYS TECHNOLOGI...|M

In [29]:
match = df0.alias("a1").join(match.alias("a2"), psf.col("a1.id") == psf.col("a2.id2"))\
    .select(psf.col("a1.job"), "a1.company", "a1.location", "a2.similarity")\
    .sort('similarity', ascending=False)

# create SQL table
match.createOrReplaceTempView("match")

In [30]:
# start SQL query

# select jobs in specific location
df = spark.sql("SELECT * FROM match WHERE location like 'New York City%'")
#df = spark.sql("SELECT * FROM match WHERE location like 'San Francisco%'")
df.show()

+--------------------+--------------------+--------------------+--------------------+
|                 job|             company|            location|          similarity|
+--------------------+--------------------+--------------------+--------------------+
|Telecommunication...|      Clarapath Inc.|New York City, Ne...| 0.04988706287672642|
|       FPGA Engineer|      Clarapath Inc.|New York City, Ne...| 0.04988706287672642|
|              python|           Synechron|New York City, Ne...|0.035548372632214956|
|Telecommunication...|      Clarapath Inc.|New York City, Ne...|0.034945310902609356|
|       web developer|      Clarapath Inc.|New York City, Ne...|0.034945310902609356|
|   Database Engineer|           Synechron|New York City, Ne...| 0.03454556273853086|
|Python Software E...|           Synechron|New York City, Ne...| 0.03454556273853086|
|           Architect|           Synechron|New York City, Ne...| 0.03454556273853086|
|Software Product ...|        Technovision|New York Ci

In [31]:
#select specific jobs
#df = spark.sql("SELECT * FROM match where job = 'computer vision engineer'")
#df = spark.sql("SELECT * FROM match where job = 'FPGA Engineer'")
df = spark.sql("SELECT * FROM match where job = 'Embedded Systems Engineer'")

df.show()

+--------------------+--------------------+--------------------+-------------------+
|                 job|             company|            location|         similarity|
+--------------------+--------------------+--------------------+-------------------+
|Embedded Systems ...|Southwest Researc...|  San Antonio, Texas|0.11408710602803729|
|Embedded Systems ...|US ARMY Ground Ve...|WARREN, Michigan ...|0.10491342174520059|
|Embedded Systems ...|              Abbott| Alameda, California|0.09937305121916463|
|Embedded Systems ...|           Talentlab|          OTTAWA, ON|0.09733485801632007|
|Embedded Systems ...|        Oculii Corp.|Beavercreek, Ohio...| 0.0955485982066785|
|Embedded Systems ...|       Kumu Networks|Sunnyvale, Califo...| 0.0954019401067412|
|Embedded Systems ...|      D3 Engineering|Rochester, New Yo...|0.09532752286044031|
|Embedded Systems ...|  NVIDIA Corporation|Santa Clara, Cali...|0.09069194568483023|
|Embedded Systems ...|         CyberCoders|  San Antonio, Texas|0